<a href="https://colab.research.google.com/github/nan-park/section4_project/blob/main/brain_tumor_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection<br>
kaggle "Brain MRI Images for Brain Tumor Detection" 데이터

In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -qq '/content/drive/MyDrive/brain_mri_images.zip'

In [4]:
# ds = tf.keras.preprocessing.image_dataset_from_directory('/content/brain_mri_images_for_brain_tumor_detection')

In [5]:
!pip install split_folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import splitfolders
input_folder = '/content/brain_mri_images_for_brain_tumor_detection'
output = '/content/output'

splitfolders.ratio(input_folder, output=output, seed=42, ratio=(0.7, 0.3))

Copying files: 253 files [00:00, 7867.49 files/s]


In [7]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory('/content/output/train')
ds_test = tf.keras.preprocessing.image_dataset_from_directory('/content/output/val')

Found 176 files belonging to 2 classes.
Found 77 files belonging to 2 classes.


In [25]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

resnet = ResNet50(weights='imagenet', include_top=False)

for layer in resnet.layers:
    layer.trainable = False

x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(32, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [31]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['acc']
)
model.fit(ds_train, batch_size=32, epochs=6, validation_data=ds_test)

Epoch 1/6
6/6 [==============================] - 5s 388ms/step - loss: 0.0090 - acc: 1.0000 - val_loss: 1.0067 - val_acc: 0.8571
Epoch 2/6
6/6 [==============================] - 1s 212ms/step - loss: 0.0053 - acc: 1.0000 - val_loss: 1.0664 - val_acc: 0.8571
Epoch 3/6
6/6 [==============================] - 2s 248ms/step - loss: 0.0221 - acc: 0.9886 - val_loss: 0.7501 - val_acc: 0.8831
Epoch 4/6
6/6 [==============================] - 1s 211ms/step - loss: 0.0079 - acc: 0.9943 - val_loss: 0.9033 - val_acc: 0.8701
Epoch 5/6
6/6 [==============================] - 1s 210ms/step - loss: 0.0103 - acc: 0.9943 - val_loss: 1.3494 - val_acc: 0.8442
Epoch 6/6
6/6 [==============================] - 2s 241ms/step - loss: 0.0053 - acc: 1.0000 - val_loss: 0.7052 - val_acc: 0.9091


In [32]:
model.evaluate(ds_test)

3/3 [==============================] - 1s 100ms/step - loss: 0.7052 - acc: 0.9091


[0.70516037940979, 0.9090909361839294]